In [1]:
# upgrade tensorflow
! pip install -q grpcio==1.24.3
! pip install -q tensorflow==2.0.0

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import brier_score_loss, roc_curve, auc
import tensorflow as tf
tf.random.set_seed(123)

In [3]:
for data in ["antenatal", "antenatal_growth", "antenatal_intrapartum"]:    
    for outcome in ['_hie', '_lapgar', '_perinataldeath', '_resus']:
        
        print("Running NN on {} for {}".format(data, outcome))
        
        # read in data
        train = pd.read_csv("data/{}{}_train.csv".format(data, outcome), index_col=0).astype('float32')
        test = pd.read_csv("data/{}{}_test.csv".format(data, outcome), index_col=0).astype('float32')
        train_y = train.pop(outcome)
        test_y = test.pop(outcome)
        
        # evaluate model
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(len(train.columns), activation='relu'),
            tf.keras.layers.Dense(len(train.columns), activation='relu'),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
        
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])
    
        # train model
        model.fit(tf.convert_to_tensor(train.to_numpy(), np.float32), tf.convert_to_tensor(train_y.to_numpy(), np.float32), epochs=20)
    
        # predict test probabilities
        y_test_pred = model.predict(tf.convert_to_tensor(test.to_numpy(), np.float32)).flatten()
    
        # calculate roc auc metric
        fpr, tpr, thresholds = roc_curve(test_y, y_test_pred)
        roc_auc = auc(fpr, tpr)
    
        # calculate brier loss for probability accuracy
        brier_loss = brier_score_loss(test_y, y_test_pred)
        
        # write out probs for delong test to obtain CIs (R)
        pd.DataFrame({"Prob" : y_test_pred, "{}{}".format(data, outcome): test_y}).to_csv("data/{}{}.NN.csv".format(data, outcome))
    
        print("ROC AUC: {}\nBrier loss: {}".format(np.round(roc_auc, 3), np.round(brier_loss, 3)))

Running NN on antenatal for _hie
Train on 26126 samples
Epoch 1/20
26126/26126 [==============================] - 2s 87us/sample - loss: 0.4264 - AUC: 0.8873
Epoch 2/20
26126/26126 [==============================] - 2s 64us/sample - loss: 0.2589 - AUC: 0.9574
Epoch 3/20
26126/26126 [==============================] - 2s 65us/sample - loss: 0.1606 - AUC: 0.9804
Epoch 4/20
26126/26126 [==============================] - 2s 67us/sample - loss: 0.1161 - AUC: 0.9879
Epoch 5/20
26126/26126 [==============================] - 2s 63us/sample - loss: 0.0942 - AUC: 0.9912
Epoch 6/20
26126/26126 [==============================] - 2s 67us/sample - loss: 0.0799 - AUC: 0.9929
Epoch 7/20
26126/26126 [==============================] - 2s 65us/sample - loss: 0.0692 - AUC: 0.9944
Epoch 8/20
26126/26126 [==============================] - 2s 63us/sample - loss: 0.0603 - AUC: 0.9954
Epoch 9/20
26126/26126 [==============================] - 2s 63us/sample - loss: 0.0535 - AUC: 0.9959
Epoch 10/20
26126/26126 [=